In [3]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import time

import numpy as np
import matplotlib.pyplot as plt
# import hoomd.htf as htf
import tensorflow.keras as keras
import tensorflow as tf


from deep_boltzmann.networks.invertible import InvNet
from deep_boltzmann.networks.invertible import split_merge_indices, SplitChannels, MergeChannels, \
                                               RealNVP, NICER, InvNet, nonlinear_transform, Scaling

/opt/anaconda3/envs/OpenMM_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/OpenMM_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/OpenMM_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/OpenMM_env/lib/py

In [4]:
# Maximum likelihood in z
def loss_z(inv_net):
    return -inv_net.log_likelihood_z_normal()

def loss_x(inv_net):
    x = inv_net.output_x
    f = inv_net.overlap_model.PotentialEnergy(x)
    #return tf.where(f>0,tf.math.log(f)-inv_net.log_det_Jzx[:,0],0)
    #return f*tf.math.exp(-inv_net.log_det_Jzx[:,0])
    return f-inv_net.log_det_Jzx[:,0]
    #return f*tf.math.exp(-inv_net.log_det_Jzx[:,0])
    

In [5]:
class OverlapInvNet(InvNet):

    def __init__(self, overlap_model, layers):
        """ Invertible net where we have an overlap function (==0 or 1) that defines p(x)
            and the prior is the uniform on the interval [a,b)"""
        self.overlap_model = overlap_model
        super().__init__(overlap_model.dim, layers, prior='normal')
        
        self.Tzx.add_loss(loss_x(self))
        self.Txz.add_loss(loss_z(self))
        
    @classmethod
    def load(cls, filename, energy_model):
        """ Loads parameters into model. Careful: this clears the whole TF session!!
        """
        from deep_boltzmann.util import load_obj
        keras.backend.clear_session()
        D = load_obj(filename)
        layerdicts = D['layers']
        layers = [eval(d['type']).from_dict(d) for d in layerdicts]
        return EnergyInvNet(energy_model, layers, prior=prior)

    def weight(self):
        """ Computes the reweighting factor
        """
        z = self.input_z
        x = self.output_x
        # compute overlap property
        f = self.overlap_model.smooth_overlap_tf(x)
        weight = tf.math.exp(self.log_det_Jzx[:, 0])
        return weight

    def sample(self, nsample=100000, temperature=1.0):
        """ Samples from prior distribution in z and produces generated x configurations

        Parameters:
        -----------
        temperature : float
            Relative temperature. Equal to the variance of the isotropic Gaussian sampled in z-space.
        nsample : int
            Number of samples

        Returns:
        --------
        sample_z : array
            Samples in z-space
        sample_x : array
            Samples in x-space
        weight_z:
            Weight of z samples
        overlap_x : array
            Overlap property of x samples
        w : array
            Weight of samples

        """

        sample_z, energy_z = self.sample_z(nsample=nsample, return_energy=True, temperature=temperature)

        sample_x, Jzx = self.transform_zxJ(sample_z)
        overlap_x = self.overlap_model.smooth_overlap_tf(sample_x).numpy()
        print(overlap_x)
        w = np.exp(Jzx)
        logw = overlap_x + energy_z + Jzx

        return sample_z, sample_x, energy_z, overlap_x, w, logw

    
    def train_z(self, x, xval=None, optimizer=None, lr=0.001, epochs=2000, batch_size=1024, verbose=1, clipnorm=None):
        if optimizer is None:
            if clipnorm is None:
                optimizer = keras.optimizers.Adam(lr=lr)
            else:
                optimizer = keras.optimizers.Adam(lr=lr, clipnorm=clipnorm)
                
        self.Txz.compile(optimizer)

        if xval is not None:
            validation_data = (xval, np.zeros_like(xval))
        else:
            validation_data = None

        hist = self.Txz.fit(x=x, validation_data=validation_data,
                            batch_size=batch_size, epochs=epochs, verbose=verbose, shuffle=True)

        return hist
    
    def train_x(self, optimizer=None, lr=0.001, batches=1,epochs=2000, batch_size=1024, verbose=1, clipnorm=None):
        if optimizer is None:
            if clipnorm is None:
                optimizer = keras.optimizers.Adam(lr=lr)
            else:
                optimizer = keras.optimizers.Adam(lr=lr, clipnorm=clipnorm)
                
        self.Tzx.compile(optimizer)

        x = self.sample_z(nsample=batches*batch_size, return_energy=False)
        hist = self.Tzx.fit(x=x,batch_size=batch_size, epochs=epochs, verbose=verbose, shuffle=True)

        return hist

    def train_both(self, x, xval=None, optimizer=None, lr=0.001, epochs=2000, batch_size=1024, verbose=1, clipnorm=None):
        if optimizer is None:
            if clipnorm is None:
                optimizer = keras.optimizers.Adam(lr=lr)
            else:
                optimizer = keras.optimizers.Adam(lr=lr, clipnorm=clipnorm)
       
        inputs = []
        outputs = []
        losses = []
        losses.append(loss_z(self))
        losses.append(loss_x(self))
        inputs.append(self.input_x)
        inputs.append(self.input_z)
        outputs.append(self.output_z)
        outputs.append(self.output_x)

        self.model = keras.models.Model(inputs=inputs, outputs=outputs)
        l = losses[0]+losses[1]
        self.model.add_loss(l)
        self.model.compile(optimizer=optimizer)
        
        if xval is not None:
            validation_data = (xval, np.zeros_like(xval))
        else:
            validation_data = None

        w = self.sample_z(nsample=x.shape[0], return_energy=False)
        hist = self.model.fit(x=[x,w], validation_data=validation_data,
                            batch_size=batch_size, epochs=epochs, verbose=verbose, shuffle=True)

        return hist

In [6]:
class Limit(object):
    def __init__(self, dim, L=1):
        self.L = L
        self.dim = dim
    
        self.tanh = tf.keras.layers.Activation('tanh')
        self.scale = tf.keras.layers.experimental.preprocessing.Rescaling(scale=self.L/2)
        self.inv_scale = tf.keras.layers.experimental.preprocessing.Rescaling(scale=2/self.L)
        self.atanh = tf.keras.layers.Activation(lambda x: tf.atanh(x))
        
    def connect_xz(self, x):
        def lambda_Jxz(x):
            J = -tf.math.log(1-(2*x/self.L)**2)
            return tf.reduce_sum(J,axis=-1)[0] * tf.ones((tf.shape(x)[0], 1))
        self.log_det_xz = keras.layers.Lambda(lambda_Jxz)(x)
        z = self.atanh(self.inv_scale(x))
        return z

    def connect_zx(self, z):
        def lambda_Jzx(z):
            J = -tf.math.log(tf.cosh(z)**2)
            return tf.reduce_sum(J,axis=-1)[0] * tf.ones((tf.shape(z)[0], 1))
        self.log_det_zx = keras.layers.Lambda(lambda_Jzx)(z)
        x = self.scale(self.tanh(z))
        return x

    @property
    def log_det_Jxz(self):
        """ Log of |det(dz/dx)| for the current batch. Format is batchsize x 1 or a number """
        return self.log_det_xz

    @property
    def log_det_Jzx(self):
        """ Log of |det(dx/dz)| for the current batch. Format is batchsize x 1 or a number """
        return self.log_det_zx


In [7]:
def invnet(dim, layer_types, overlap_model=None, channels=None, L=1.0,
           nl_layers=2, nl_hidden=100, nl_activation='relu',
           nl_activation_t='relu',scale=None, prior='normal'):
    """
    layer_types : str
        String describing the sequence of layers. Usage:
            N NICER layer
            R RealNVP layerl
            S Scaling layer
        Splitting and merging layers will be added automatically
    overlap_model : Overlap model class
        Class with overlap_tf() and dim
    channels : array or None
        Assignment of dimensions to channels (0/1 array of length ndim)
    nl_layers : int
        Number of hidden layers in the nonlinear transformations
    nl_hidden : int
        Number of hidden units in each nonlinear layer
    nl_activation : str
        Activation functions used in the nonlinear layers
    scale : None or float
        If a scaling layer is used, fix the scale to this number. If None, scaling layers are trainable
    """
    # fix channels
    channels, indices_split, indices_merge = split_merge_indices(dim, nchannels=2, channels=channels)

    # augment layer types with split and merge layers
    split = False
    tmp = ''
    for ltype in layer_types:
        if (ltype == 'S' or ltype == 'L') and split:
            tmp += '>'
            split = False
        if (ltype == 'N' or ltype == 'R') and not split:
            tmp += '<'
            split = True
        tmp += ltype
    if split:
        tmp += '>'
    layer_types = tmp
    print(layer_types)

    # prepare layers
    layers = []

#     reg = tf.keras.regularizers.l2(.01) #?
    reg = None
    for ltype in layer_types:
        if ltype == '<':
            # split into two x channels
            layers.append(SplitChannels(dim, nchannels=2, channels=channels))
        if ltype == '>':
            # merge into one z channel
            layers.append(MergeChannels(dim, nchannels=2, channels=channels))
        if ltype == 'N':
            M1 = nonlinear_transform(indices_split[1].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation)
            M2 = nonlinear_transform(indices_split[0].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation)
            layers.append(NICER([M1, M2]))
        elif ltype == 'R':
            S1 = nonlinear_transform(indices_split[1].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation, init_outputs=0,
                                     activity_regularizer=reg)
            T1 = nonlinear_transform(indices_split[1].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation_t)
            S2 = nonlinear_transform(indices_split[0].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation, init_outputs=0,
                                     activity_regularizer=reg)
            T2 = nonlinear_transform(indices_split[0].size, nlayers=nl_layers, nhidden=nl_hidden,
                                     activation=nl_activation_t)
            layers.append(RealNVP([S1, T1, S2, T2]))
        elif ltype == 'L':
            layers.append(Limit(dim, L=L))
        elif ltype == 'S':
            # scaling layer
            if scale is None:
                scaling_factors = None
            else:
                scaling_factors = scale * np.ones((1, dim))
            layers.append(Scaling(dim, scaling_factors=scaling_factors, trainable=(scale is None)))

    if overlap_model is None:
        inv_net = InvNet(dim, layers, prior='normal')
    else:
        inv_net = OverlapInvNet(overlap_model, layers)
    
    inv_net.reg = reg
    return inv_net

In [20]:
class OneWaterTest(object):
    def __init__(self,N):
        """ N: number of particles
            L: Box length
            sigma: Particle size"""
        self.N = N
        self.dim = 3*self.N # dim = 2
        pdb = PDBFile('water.pdb')
        self.forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
        self.modeller = Modeller(pdb.topology, pdb.positions)
        # modeller.deleteWater()
        self.modeller.addHydrogens(self.forcefield)

        self.system = self.forcefield.createSystem(self.modeller.topology, nonbondedMethod=NoCutoff, 
                                                   constraints=None, rigidWater=False)
        # modeller.positions[0] = Quantity(Vec3(0,0,0), unit=nanometer)
        self.integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
        self.simulation = Simulation(self.modeller.topology, self.system, self.integrator)
        self.simulation.context.setPositions(self.modeller.positions)
        # print(pdb.positions)
        self.state = self.simulation.context.getState(getEnergy=True)
        self.energy = self.state.getPotentialEnergy()
        print(self.energy)
        
    def PotentialEnergy(self, x=None):
        # x has to be an 2D array of 6*3
        if x is not None:
            print(f'x is not None')
            temp_positions = [None]*self.N
            for ii in range(self.N):
                xi = x[ii]
                temp_positions[ii] = Quantity(Vec3(xi[0], xi[1], xi[2]), unit=nanometer)
            print(temp_positions)
            self.simulation.context.setPositions(temp_positions)
            
            self.state = self.simulation.context.getState(getEnergy=True,getPositions=True)
            self.energy = self.state.getPotentialEnergy()
            print(self.state.getPositions())

        return self.energy
        

In [21]:
Water = OneWaterTest(3)

8.309070587158203 kJ/mol


In [22]:
Water.PotentialEnergy(x=[[0,0,0],[0,1,0],[1,1,0]]) / kilojoule_per_mole

x is not None
[Quantity(value=Vec3(x=0, y=0, z=0), unit=nanometer), Quantity(value=Vec3(x=0, y=1, z=0), unit=nanometer), Quantity(value=Vec3(x=1, y=1, z=0), unit=nanometer)]
[Vec3(x=0.0, y=0.0, z=0.0), Vec3(x=0.0, y=1.0, z=0.0), Vec3(x=1.0, y=1.0, z=0.0)] nm


586334.0

In [23]:
def act(x):
    return -tf.nn.relu(x)

In [24]:
# reset model
#bg = invnet(model.dim, 'LRRRRRRRRRRRR', overlap_model=model, nl_layers=4, nl_hidden=200, #100
bg = invnet(Water.dim, 'RRRR', overlap_model=Water, nl_layers=2, nl_hidden=10, #200, #100
#            nl_activation='tanh', nl_activation_t='tanh', L=L)
           nl_activation='tanh', nl_activation_t='relu')

<RRRR>
x is not None
[Quantity(value=Vec3(x=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_11')>, y=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_12')>, z=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_13')>), unit=nanometer), Quantity(value=Vec3(x=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_15')>, y=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_16')>, z=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_17')>), unit=nanometer), Quantity(value=Vec3(x=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_19')>, y=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_20')>, z=<KerasTensor: shape=() dtype=float32 (created by layer 'tf.__operators__.getitem_21')>), unit=nanometer)]


ValueError: in method Context_setPositions, argument 2 could not be converted to type std::vector< Vec3,std::allocator< Vec3 > > const &

In [29]:
print(Water.dim)

9
